In [1]:
import ipywidgets as widgets
from SPARQLWrapper import SPARQLWrapper, JSON

class HealthDM(object):
    profile = None
    
    @classmethod
    def setProfile(cls, profile):
        cls.profile = profile
        
    @classmethod
    def clearConcepts(cls):
        cls.profile.children = tuple([])

    @classmethod
    def addConcept(cls, concept, code):
        wt = widgets.Text()
        wt.value = code
        cls.profile.children = tuple(list(cls.profile.children) + [wt])
        cls.profile.set_title(len(cls.profile.children)-1, concept)

    @classmethod
    def findMeshCode(cls, heading):
        sparql = SPARQLWrapper("http://id.nlm.nih.gov/mesh/sparql")
        sparql.setQuery("""
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
PREFIX mesh2015: <http://id.nlm.nih.gov/mesh/2015/>
PREFIX mesh2016: <http://id.nlm.nih.gov/mesh/2016/>
PREFIX mesh2017: <http://id.nlm.nih.gov/mesh/2017/>
PREFIX mesh2018: <http://id.nlm.nih.gov/mesh/2018/>

SELECT ?d
FROM <http://id.nlm.nih.gov/mesh>
WHERE {
    ?d a meshv:Term .
    {?d meshv:prefLabel ?l
     FILTER (lcase(str(?l)) = lcase('""" + heading + """'))}
    UNION
    {?d meshv:altLabel ?l
     FILTER (lcase(str(?l)) = lcase('""" + heading + """'))}
} 
ORDER BY ?d
""")
        sparql.setReturnFormat(JSON)
        results = sparql.query().convert()
    
        code = "none"
        if len(results["results"]["bindings"]) > 0:
            code = results["results"]["bindings"][0]["d"]["value"]
            
        cls.addConcept(heading, code)
    
        return "#mesh_heading#" + heading + "#tree_number#" + code


profile = widgets.Accordion(children=[])

HealthDM.setProfile(profile)

profile

Accordion()

# Case 1

=== VP ===

= more_information


=== Characters ===

= Nurse
![Nurse](nurse-1529355567.png)

= Patient
* 23 years old

=== Start ===

:Nurse: Doctor, please come to the Emergency room, we have a difficult case there.

= First

= Second

=== More Information ===

His colleagues found him sited inside the forklift, {without signs of trauma}(no trauma). After they took him out and laid him down, the patient woke up but still complained of {dizziness} and could not stand up.
There was {no blood} at the scene, neither other signs of trauma. The patient had {no previous syncope episodes}, he {drinks alcohol occasionally}, and {smokes 10 cigarettes a day}.


{Myocardial Infarction}

+ escolha

* item
  * desvia para ->titulo
  * desvia para ->lugar_nenhum
  * desvia para ->Start.First
  * desvia para ->First

==== titulo ====

{Aortic Dissection}

# Case 29

Patient a man 55 years old; rather fat; subject to frequent attacks of winter {cough}, with {asthmatic} tendency. For seven years the heart had been noticeably weak and irregular. Pulse 80; first sound valvular. Apex beat an inch and a half directly below left nipple; no murmurs. No previous rheumatism. Several years ago there was sudden and complete {loss of memory}, the same questions being repeated as soon as answered. The expression was at the time rather vacant; the pupils were equal and responded to light; there was no motor paralysis. The {amnesia} lasted all day, disappearing the following morning. The pulse remained 50 for two days. The patient had been previously very {anemic}, and had had much fatigue and anxiety, with digestive disturbance. The urine always remained normal. In the following years there were occasional attacks of transient numbness in the left arm and leg, and sometimes faint turns with pallor and irregular, feeble pulse. Headache was a frequent symptom; dyspnea on exertion, impaired appetite, and insomnia were constant. There was no apparent loss of flesh. In 1897 life was endangered for two weeks by edema of both lungs, supervening on an attack of {bronchitis}. In the subsequent years the condition improved somewhat, so that the patient could walk half a mile or more and was able to attend to considerable business. In autumn of 1904, be had several attacks of bronchitis, and, finally, an attack of complete {hemiplegia} resulted fatally in twenty-four hours without recovery of consciousness. Respiration was of the Cheyne-Stokes type, and later stertorous.